<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_07/class_007_image_processing_smart_trash_bin_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 193MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 224
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    img = cv2.imread(name)
    resized = cv2.resize(img, (Img_Size,Img_Size))
    X.append(resized)
m = len(Y)
print(n,m)

2527 2527


In [9]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [10]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [11]:
df=pd.DataFrame(label,columns=['target'])

In [12]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [13]:
y = pd.get_dummies(df['target'])
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
#y

In [14]:
#label

In [15]:
n =len(img_name)
data = np.empty((n, 224, 224, 3))

In [16]:
# https://pythontutorials.eu/deep-learning/transfer-learning/

In [17]:
from keras.applications.mobilenet_v2 import preprocess_input
from skimage.transform import resize
i = 0
for im in X:
    im = preprocess_input(im)
    im = resize(im, output_shape=(224, 224))
    data[i] = im
    i += 1

In [18]:
n

2527

In [19]:
y.values

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [20]:
n =len(img_name)
m = len(tipos)

'''
labels_tf = np.empty(n, dtype=int)
for idx, yi in enumerate(label):
  labels_tf[idx] = yi
'''
idx = 0
labels_tf = np.empty([n,m], dtype=int)
for yi in y.values:
  labels_tf[idx] = yi
  idx = idx + 1



In [21]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(weights='imagenet')

14536120/14536120 [==============================] - 0s 0us/step


In [22]:
from keras.applications.mobilenet_v2 import decode_predictions

predictions = model.predict(data)
k = 0
for decoded_prediction in decode_predictions(predictions, top=1):
  
  for name, desc, score in decoded_prediction:
      if(k % 200 ==0):
        print('- {} ({:.2f}%%) {}'.format(desc, 100 * score,img_name[k]))
  k = k +1

35363/35363 [==============================] - 0s 0us/step
- envelope (27.77%%) cardboard1.jpg
- mailbag (27.79%%) cardboard201.jpg
- velvet (10.09%%) cardboard401.jpg
- ballpoint (13.35%%) glass198.jpg
- spotlight (96.14%%) glass398.jpg
- water_bottle (24.05%%) metal97.jpg
- ashcan (71.91%%) metal297.jpg
- syringe (20.62%%) paper87.jpg
- shopping_cart (10.26%%) paper287.jpg
- hand-held_computer (23.41%%) paper487.jpg
- water_bottle (18.48%%) plastic93.jpg
- water_bottle (60.90%%) plastic293.jpg
- plastic_bag (69.38%%) trash11.jpg


In [23]:
from keras.layers import Dense
D = len(tipos)
model_output = Dense(D, activation='softmax')

In [24]:
model_output = model_output(model.layers[-2].output)

In [25]:
from keras import Model

model_input = model.input
model_model = Model(inputs=model_input, outputs=model_output)

In [26]:
for layer in model_model.layers[:-1]:
    layer.trainable = False

In [27]:
model_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
model_model.fit(x=data, y=labels_tf, epochs=20, verbose=2)

Epoch 1/20
79/79 - 6s - loss: 1.0225 - accuracy: 0.6150 - 6s/epoch - 78ms/step
Epoch 2/20
79/79 - 3s - loss: 0.5576 - accuracy: 0.8045 - 3s/epoch - 42ms/step
Epoch 3/20
79/79 - 3s - loss: 0.4449 - accuracy: 0.8449 - 3s/epoch - 42ms/step
Epoch 4/20
79/79 - 3s - loss: 0.3755 - accuracy: 0.8678 - 3s/epoch - 43ms/step
Epoch 5/20
79/79 - 3s - loss: 0.3346 - accuracy: 0.8841 - 3s/epoch - 44ms/step
Epoch 6/20
79/79 - 3s - loss: 0.2928 - accuracy: 0.9050 - 3s/epoch - 42ms/step
Epoch 7/20
79/79 - 3s - loss: 0.2599 - accuracy: 0.9232 - 3s/epoch - 42ms/step
Epoch 8/20
79/79 - 3s - loss: 0.2319 - accuracy: 0.9315 - 3s/epoch - 42ms/step
Epoch 9/20
79/79 - 3s - loss: 0.2098 - accuracy: 0.9422 - 3s/epoch - 42ms/step
Epoch 10/20
79/79 - 3s - loss: 0.1952 - accuracy: 0.9505 - 3s/epoch - 42ms/step
Epoch 11/20
79/79 - 3s - loss: 0.1760 - accuracy: 0.9569 - 3s/epoch - 42ms/step
Epoch 12/20
79/79 - 3s - loss: 0.1654 - accuracy: 0.9628 - 3s/epoch - 42ms/step
Epoch 13/20
79/79 - 3s - loss: 0.1515 - accuracy: